---

# Python dependences

In [4]:
!pip install numpy==1.26.4 #need it for the old version of pomegranate

In [5]:
!pip install pomegranate==0.15.0

---

# EXERCISE 1: Burglary network

Implement the *Prior Sampling* algorithm to do approximate inference on last week's Burglary network.

<img src='https://drive.google.com/uc?id=11xqbchd4TCFSzdLVxNgr4SoeGfrSoqGO'>

Verify that the algorithm can correctly approximate the probability $P(j, m, a, \neg b, \neg e) = 0.00063$

Try different numbers of samples (e.g. $N = 10, 100, 1000, 10000$) and compare the results.

---

In [20]:
import numpy as np
import random as rnd

# defining states of the variables
t, f = 0, 1

# Defining CPTs for probability network variables
P_B = np.array([0.001,0.999])
P_E = np.array([0.002,0.998])
P_A_be = np.array([[[0.95, 0.94], [0.29, 0.001]], [[0.05, 0.06], [0.71, 0.999]]])
P_J_a = np.array([[0.9, 0.05], [0.1, 0.95]])
P_M_a = np.array([[0.7, 0.01], [0.3, 0.99]])

# variables in topological order
var = ["B", "E", "A", "J", "M"]

# related probability distributions
prob = {"B": P_B, "E": P_E, "A": P_A_be, "J": P_J_a, "M": P_M_a}

# parents
par = {"B": [], "E": [], "A": ["B", "E"], "J": ["A"], "M": ["A"]}

# values of the variables in the sample, initialized by default to false
val = {"B": f, "E": f, "A": f, "J": f, "M": f}

# defining sample generator
def samplegen(Pdist, Parents = []):
    assert len(Parents) < len(Pdist.shape)
    if rnd.random() < Pdist[t][tuple(Parents)]:
        return t
    return f

# get values of parents variables in the current sample
def parents(X):
    return [val[i] for i in par[X]]

# defining counters for calculating approximate probability by sampling
n_samples = 1000000
counter_true_samples = 0

# calculating approximate probability P(j,m,a,¬b,¬e)
for i in range (n_samples):
    # generating sample
    for v in var:
        val[v] = samplegen(prob[v], parents(v))

    # checking sample
    if ['f' if val[x] else 't' for x in var] == ["f","f","t","t","t"]:
        counter_true_samples += 1

# printing approximate probability
print(counter_true_samples)
print("P(j,m,a,¬b,¬e) = ", counter_true_samples / n_samples)

632
P(j,m,a,¬b,¬e) =  0.000632


---

# EXERCISE 2: Pomegranate for Day 2

Use <tt>pomegranate</tt> to calculate the filtered probability of rain on Day 2 when we see an umbrella on Day 1 and Day 2.

What is the filtered probability of rain on Day 2 when we see <tt>not umbrella</tt> on Day 1? (But we still see it in Day 2)

How about if we just have no information about Umbrellas on Day 1 (But we still see the umbrella in Day 2)?

In [7]:
#from pomegranate import DiscreteDistribution
import numpy as np
from pomegranate import *

# Variables are RainN and UmbrellaN+1 for N = 0, 1, ...
# We have a prior for Rain0, two values 'y'es and 'n'o:

# Prior distribution for Rain0
Rain0 = DiscreteDistribution({'y': 0.5, 'n': 0.5})

# definition of CPT for Transition model P(RainN+1 | RainN)
TransitionModel_CPT = [['y', 'y', 0.7], ['y', 'n', 0.3], ['n', 'y', 0.3], ['n', 'n', 0.7]]

# creating of CPTs for nodes Rain1, Rain2 and Rain3 based on the transition model
Rain1 = ConditionalProbabilityTable(TransitionModel_CPT, [Rain0])
Rain2 = ConditionalProbabilityTable(TransitionModel_CPT, [Rain1])
Rain3 = ConditionalProbabilityTable(TransitionModel_CPT, [Rain2])

# definition of CPT for Sensor model P(UmbrellaN | RainN)
SensorModel_CPT = [['y', 'y', 0.9], ['y', 'n', 0.1], ['n', 'y', 0.2], ['n', 'n', 0.8]]

# creating of CPTs for nodes Umbrella1 and Umbrella2 based on the sensor model
Umbrella1 = ConditionalProbabilityTable(SensorModel_CPT, [Rain1])
Umbrella2 = ConditionalProbabilityTable(SensorModel_CPT, [Rain2])

# The whole network has five nodes:
s1 = Node(Rain0, name="Rain0")
s2 = Node(Rain1, name="Rain1")
s3 = Node(Umbrella1, name="Umbrella1")
s4 = Node(Rain2, name="Rain2")
s5 = Node(Umbrella2, name="Umbrella2")
s6 = Node(Rain3, name="Rain3")

# Create a network that includes nodes and edges between them:
model = BayesianNetwork("Umbrella Network")
model.add_states(s1, s2, s3, s4, s5, s6)
model.add_edge(s1, s2)
model.add_edge(s2, s3)
model.add_edge(s2, s4)
model.add_edge(s4, s5)
model.add_edge(s4, s6)

# Fix the model structure
model.bake()

In [8]:
# computing the filtered probability of rain on Day 2 when we see an umbrella on Day 1 and Day 2
scenario = [[None, None, 'y', None, 'y', None]]
results =  model.predict_proba(scenario)
print("Rain at day0", results[0][0].items())
print("Rain at day1", results[0][1].items())
print("Rain at day2", results[0][3].items())
print("Rain at day3", results[0][5].items())

Rain at day0 (('y', 0.653342816500711), ('n', 0.34665718349928903))
Rain at day1 (('y', 0.8833570412517778), ('n', 0.11664295874822224))
Rain at day2 (('y', 0.8833570412517777), ('n', 0.11664295874822232))
Rain at day3 (('y', 0.6533428165007111), ('n', 0.346657183499289))


In [9]:
# computing the filtered probability of rain on Day 2 when we see not_umbrella on Day 1 and an umbrella on Day 2
scenario = [[None, None, 'n', None, 'y', None]]
results =  model.predict_proba(scenario)
print("Rain at day0", results[0][0].items())
print("Rain at day1", results[0][1].items())
print("Rain at day2", results[0][3].items())
print("Rain at day3", results[0][5].items())

Rain at day0 (('y', 0.36952141057934534), ('n', 0.6304785894206547))
Rain at day1 (('y', 0.1738035264483631), ('n', 0.8261964735516368))
Rain at day2 (('y', 0.7027707808564231), ('n', 0.2972292191435768))
Rain at day3 (('y', 0.5811083123425691), ('n', 0.4188916876574308))


In [10]:
# computing the filtered probability of rain on Day 2 when there are no information about Umbrellas on Day 1, but we sill see an umbrella in Day 2
scenario = [[None, None, None, None, 'y', None]]
results =  model.predict_proba(scenario)
print("Rain at day0", results[0][0].items())
print("Rain at day1", results[0][1].items())
print("Rain at day2", results[0][3].items())
print("Rain at day3", results[0][5].items())

Rain at day0 (('y', 0.5509090909090909), ('n', 0.44909090909090926))
Rain at day1 (('y', 0.6272727272727272), ('n', 0.3727272727272729))
Rain at day2 (('y', 0.8181818181818179), ('n', 0.1818181818181821))
Rain at day3 (('y', 0.6272727272727271), ('n', 0.3727272727272729))


In [11]:
# computing filtered probability by imposing probability of rain on Day 0 is 1 (to compare the results in the Excel file)
scenario = [['y', None, 'y', None, 'y', None]]
results =  model.predict_proba(scenario)
print("Rain at day0", results[0][0]) # no .items() because we are sure that it rained on Day 0
print("Rain at day1", results[0][1].items())
print("Rain at day2", results[0][3].items())
print("Rain at day3", results[0][5].items())

# NOTE: even if we impose the probability of rain on Day 0 is 1, pomegranate calculate the probability of Day 1 applying
# smoothing on the results, so Excel results and Pomegranate results differ.

Rain at day0 y
Rain at day1 (('y', 0.9464402351404309), ('n', 0.05355976485956919))
Rain at day2 (('y', 0.8994121489222726), ('n', 0.10058785107772737))
Rain at day3 (('y', 0.6597648595689091), ('n', 0.340235140431091))
